In [165]:
import polars as pl

In [166]:
df_preppet = pl.read_parquet('kjoretoyinfo_preppet.parquet')

In [167]:
df_preppet

tekn_reg_f_g_n,tekn_reg_eier_dato,tekn_aksler_drift,tekn_merke,tekn_modell,tekn_drivstoff,tekn_neste_pkk,er_elektrisk,tekn_farge
date,date,i64,i64,str,str,date,bool,str
2021-10-13,2021-10-13,2,3900,"""VITO""","""2""",2025-10-13,false,"""05"""
2020-01-29,2020-01-29,2,5750,"""CADDY""","""2""",2026-01-29,false,"""06"""
2023-11-29,2023-11-29,1,2600,"""Mustang Mach-E…","""5""",2027-11-29,true,"""06"""
2020-01-27,2020-01-27,2,5750,"""CADDY""","""2""",2024-01-27,false,"""01"""
2020-05-19,2020-05-19,1,5750,"""CRAFTER""","""2""",2024-05-19,false,"""01"""
…,…,…,…,…,…,…,…,…
2021-10-21,2021-10-21,2,3900,"""GLE 350 de 4MA…","""2,5""",2025-10-21,false,"""03"""
2023-09-01,2023-09-01,0,320,"""Adora 573 PT""",null,null,null,"""01"""
2020-12-29,2020-12-29,1,4000,"""MG ZS EV""","""5""",2024-12-29,true,"""02"""


In [168]:
#7. Hvor mange elbiler (drivstofftype 5) ble førstegangsregistrert i 2022?
elbiler_2022 = df_preppet.filter(
    (pl.col("er_elektrisk")) & 
    (pl.col("tekn_reg_f_g_n").dt.year() == 2022)
)

antall_elbiler_2022 = elbiler_2022.height

In [169]:
print(antall_elbiler_2022)

37249


In [170]:
#8.Hvor mange prosent av personbilene som ble solgt i 2022 var elbiler?
elbiler_2022 = df_preppet.filter(
    (pl.col("er_elektrisk")) & 
    (pl.col("tekn_reg_eier_dato").dt.year() == 2022)
)

antall_elbiler_solgt_2022 = elbiler_2022.height

In [171]:
alle_biler_2022 = df_preppet.filter(
  (pl.col("tekn_reg_eier_dato").dt.year() == 2022)
)

antall_biler_solgt_2022 = alle_biler_2022.height

In [172]:
prosent_elbiler_solgt_2022 = (antall_elbiler_solgt_2022 / antall_biler_solgt_2022) * 100

In [173]:
print(prosent_elbiler_solgt_2022)

25.194972733136566


In [174]:
#15. Hvilke bilmerker var de fem mest populære i 2022?
df_2022 = df_preppet.filter(
     (pl.col("tekn_reg_eier_dato").dt.year() == 2022)
)

In [175]:
top_5_popular_brands=df_2022.group_by("tekn_modell").agg(
    pl.col("tekn_modell").count().alias("antall_modeller")
).sort("antall_modeller", descending=True).head(5)

In [180]:
print(top_5_popular_brands)

shape: (5, 2)
┌─────────────────┬─────────────────┐
│ tekn_modell     ┆ antall_modeller │
│ ---             ┆ ---             │
│ str             ┆ u32             │
╞═════════════════╪═════════════════╡
│ Model Y         ┆ 8401            │
│ ID.4 GTX 220 KW ┆ 3676            │
│ XC40            ┆ 3017            │
│ TYSSE           ┆ 2873            │
│ IONIQ5          ┆ 2715            │
└─────────────────┴─────────────────┘


In [185]:
#16. Hva var den mest populære fargen for de fem mest populære bilmerkene i 2022 (førstegangsregistrert 2022)
top_5_brands_list = top_5_popular_brands['tekn_modell'].to_list()

In [186]:
filtered_df = df_2022.filter(pl.col("tekn_modell").is_in(top_5_brands_list))

In [187]:
grouped_colors_df = filtered_df.group_by(["tekn_modell", "tekn_farge"]).agg(
    pl.col("tekn_farge").count().alias("antall_farger")
)

In [188]:
most_popular_colors = grouped_colors_df\
    .sort("antall_farger", descending=True)\
    .group_by("tekn_modell")\
    .agg(
        pl.first("tekn_farge").alias("mest_populær_farge"),
        pl.first("antall_farger").alias("antall_farger")
    )


In [189]:
print(most_popular_colors)

shape: (5, 3)
┌─────────────────┬────────────────────┬───────────────┐
│ tekn_modell     ┆ mest_populær_farge ┆ antall_farger │
│ ---             ┆ ---                ┆ ---           │
│ str             ┆ str                ┆ u32           │
╞═════════════════╪════════════════════╪═══════════════╡
│ XC40            ┆ 06                 ┆ 1029          │
│ Model Y         ┆ 06                 ┆ 3431          │
│ IONIQ5          ┆ 04                 ┆ 729           │
│ TYSSE           ┆ 05                 ┆ 2873          │
│ ID.4 GTX 220 KW ┆ 05                 ┆ 919           │
└─────────────────┴────────────────────┴───────────────┘


In [2]:
#Hvor mange gule kjøretøy ble det solgt i Mai 2022?

import pandas as pd

# Les inn datasettet
dataset = pd.read_parquet('kjoretoyinfo_preppet.parquet')


In [3]:
dataset['tekn_reg_f_g_n'] = pd.to_datetime(dataset['tekn_reg_f_g_n'])


In [6]:
mai_2022_gule_kjoretoy = dataset[(dataset['tekn_reg_f_g_n'].dt.month == 5) & (dataset['tekn_reg_f_g_n'].dt.year == 2022) & (dataset['tekn_farge'] == 'Gul')]


In [7]:
# Tell antallet rader i det filtrerte datasettet
antall_gule_kjoretoy = len(mai_2022_gule_kjoretoy)


In [8]:
# Skriv ut resultatet
print("Antall gule kjøretøy solgt i Mai 2022:", antall_gule_kjoretoy)


Antall gule kjøretøy solgt i Mai 2022: 0


In [ ]:
#Hvilken farge var den mest populære for traktorer i 2022 se teknisk kode.csv
# Grupper dataene etter farge og tell antall forekomster av hver farge
farge_count = dataset['tekn_farge'].value_counts()
